# EECS531 - A3
## Tristan Maidment (tdm47)

#### Goal
This is the first assignment that requires a neural network to be implemented. Keras provides a very hands off approach for defining a neural network.  Although this is useful for helping build the intuition of how to structure a neural network, I would like a full understanding of how it works, e.g. defining one by hand.

In [1]:
import numpy as np
import mnist
import cv2

The first step is to download the mnist dataset.

In [2]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()

test_images = mnist.test_images()
test_labels = mnist.test_labels()

The first thing to define is the classifier functions used at each layer.  For the purpose of speed, the main classifier used will be the ReLu function.

In [3]:
def relu(x):
    return np.maximum(x, 0)

def d_relu(x):
    return np.maximum(x, 0 ) * x

since we are using a numpy implentation and everything will be super slow, pooling will save my life

In [4]:
#only supports pool sizes that fit in the img
def max_pool(img, value):
    h, w = img.shape
    h_pool, w_pool = int(h/value), int(w/value)
    pool = np.empty((h_pool, w_pool))
    for i in range(h_pool):
        for j in range(w_pool):
            pool[i, j] = img[i*value:(i+1)*value,j*value:(j+1)*value].max()
    return pool

define functions used to define the weights of each layer

In [11]:
# x kernels of y size
def conv_weights(x, y):
    c = []
    for i in range(x):
        c.append(np.random.randn(y, y)) #init with random weights
    return np.array(c)

# flatten layer weights
# x - input dimensions
# y - output dimensions
def flat_weights(x, y):
    return np.random.randn(x, y)

def bias_weights(x):
    return np.random.randn(x)

layer feed forwards

In [15]:
def conv_feed_forward(img, kernels):
    c = [] #
    c_l = [] #linear
    for i in range(len(kernels)):
        # conv layer
        img_conv = cv2.filter2D(img, -1, kernels[i])
        c_l.append(img_conv)
        img_relu = relu(img_conv)
        #img_pool = max_pool(img_relu, 2)
        c.append(img_relu)
    return np.array(c), np.array(c_l)

def flatten_feed_forward(convs, weights):
    f = []
    for i in range(len(convs)):
        f.append(np.ravel(convs[i]))
    reshape = np.expand_dims(np.hstack(f), axis=0)
    flat = np.dot(reshape, weights)
    return relu(flat), reshape

def dense_feed_forward(layer, weights, bias):
    z = np.add(np.dot(layer, weights), bias)
    #z = np.dot(layer, weights)
    return relu(z), z

# serious switch this to cross entropy
def cost_feed_forward(layer, label):
    return np.multiply(np.square(np.sum(layer - label)), 0.5)

layer back projection

In [16]:
def cost_back_propagation(layer, label):
    return np.multiply(2, np.subtract(layer, label)) #gradient of the 

def dense_back_propagation(inner, prev_layer):
    #print(inner.shape, prev_layer.shape)
    return np.outer(d_relu(inner), prev_layer) #might have to be dot

In [17]:
def cost_likelihood(vector):
    return np.argmax(vector)

def one_hot_label(label):
    ret = np.zeros(10)
    ret[label] = 1
    return ret

shallow neural network definition

In [18]:
epochs = 3
#w_l0 = conv_weights(6, 3) # 6 3x3 kernels 
w_0 = flat_weights(784, 256) # flatten to a 256 hidden layer
w_1 = flat_weights(256, 10) # flatten to a 10 node output layer
b_0 = bias_weights(256)
b_1 = bias_weights(10)
cost = 0
alpha = 0.01

In [19]:
for epoch in range(epochs):
        for i in range(len(train_images)):
            img = train_images[i]
            label = train_labels[i]
            
            img_input = img.ravel()
            
            a_0, z_0 = dense_feed_forward(img_input, w_0, b_0)
            a_1, z_1 = dense_feed_forward(a_0, w_1, b_1)
            
            cost += cost_feed_forward(a_1, one_hot_label(label))
            print("Estimated value:",cost_likelihood(a_1), "Actual Value:", label)
            
            delta_cost = cost_back_propagation(a_1, one_hot_label(label))
            #print(delta_cost.shape)
            delta_1 = dense_back_propagation(z_1, a_0)
            #print(delta_1.shape, w_1.shape)
            
            delta_0 = dense_back_propagation(z_0, w_0)
            #print(delta_0.shape, w_0.shape)
            
            b_1 += np.multiply(z_1, delta_cost)
            b_0 += np.multiply(z_0, delta_cost)
            
            w_1a = np.multiply(delta_1.T, delta_cost)
            print("w_1a", np.mean(w_1a))
            w_1 += w_1a#np.multiply(alpha, w_1a)
            
            w_0a = np.multiply(delta_0.T, delta_cost)
            print("w_0a", np.mean(w_0a))
            w_0 += w_0a#np.multiply(alpha, w_0a)
            

Estimated value: 5 Actual Value: 5


ValueError: operands could not be broadcast together with shapes (256,) (10,) 